# Using dataobject-based classes for creating common datatypes

In [1]:
from recordclass import dataobject
from recordclass._linkedlist import linkedlist
import sys

## LinkedList

In [ ]:
class LinkedItem(dataobject, fast_new=True):
    val: object
    next: 'LinkedItem'

class LinkedList(dataobject, deep_dealloc=True):
    start: LinkedItem = None
    end: LinkedItem = None
        
    def append(self, val):
        link = LinkedItem(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link
        
#     def __del__(self):
#         curr = self.start
#         while curr is not None:
#             next = curr.next
#             curr.next = None
#             curr = next

    def __iter__(self):
        return IterLinkedList(self)

class IterLinkedList(dataobject, fast_new=True):
    node: LinkedItem
        
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [ ]:
def make_llist(N):
    ll = LinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [ ]:
class LinkedItem2:
    __slots__ = 'val', 'next'

    def __init__(self, val, next):
        self.val = val
        self.next = next
    
class LinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = LinkedItem2(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self)

class IterLinkedList2:
    __slots__ = 'node',
    
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val

In [ ]:
def make_llist2(N):
    ll = LinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [ ]:
N = 1000000
Mb = 1000000

In [ ]:
%time ll1 = make_llist(N)
%timeit make_llist(N)

In [ ]:
%time s1 = sum(ll1)
%timeit sum(ll1)
print(s1)

In [ ]:
print(sys.getsizeof(ll1.start), sys.getsizeof(ll1))
M1 = sys.getsizeof(ll1) + N * sys.getsizeof(ll1.start)
print('Memory footprint: %.2f Мб' % (M1/1000000))
# del ll1

In [ ]:
%time ll2 = make_llist2(N)
%timeit make_llist2(N)

In [ ]:
%time s2 = sum(ll2)
%timeit sum(ll2)
print(s2)

In [ ]:
print(sys.getsizeof(ll2.start), sys.getsizeof(ll2))
M2 = sys.getsizeof(ll2) + N * sys.getsizeof(ll2.start)
print('Memory footprint: %.2f Мб' % (M2/1000000))
# del ll2

In [ ]:
print(100*M1/M2)

In [ ]:
def make_llist3(N):
    ll = linkedlist()
    for i in range(N):
        ll.append(i)
    return ll

In [ ]:
%time ll3 = make_llist3(N)
%timeit make_llist3(N)

In [ ]:
%time s3 = sum(ll3)
%timeit sum(ll3)
print(s3)

In [ ]:
M3 = sys.getsizeof(ll3) + N * sys.getsizeof(ll3.start)
print('Memory footprint: %.2f Мб' % (M3/1000000))
# del ll3

## Double Linked List

In [ ]:
class DLinkedItem(LinkedItem, fast_new=True):
#     val: object
#     next: 'DLinkedItem'
    prev: 'DLinkedItem'

class DLinkedList(LinkedList, deep_dealloc=True):
#     start: DLinkedItem = None
#     end: DLinkedItem = None
        
    def append(self, val):
        link = DLinkedItem(val, None, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        link.prev = self.end
        self.end = link

    def __iter__(self):
        return IterLinkedList(self)

class IterLinkedList(dataobject, fast_new=True):
    node: DLinkedItem
        
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [ ]:
def make_dllist(N):
    ll = DLinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [ ]:
class DLinkedItem2(LinkedItem2):
    __slots__ = 'prev',

    def __init__(self, val, next, prev):
        self.val = val
        self.next = next
        self.prev = prev
    
class DLinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = DLinkedItem2(val, None, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        link.prev = self.end
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self)

# class DIterLinkedList2:
#     __slots__ = 'node',
    
#     def __init__(self, ll):
#         self.node = ll.start
    
#     def __next__(self):
#         node =  self.node
#         if node is None:
#             raise StopIteration
        
#         val = node.val
#         self.node = node.next
#         return val

In [ ]:
def make_dllist2(N):
    ll = DLinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [ ]:
%time dll1 = make_dllist(N)
%timeit make_dllist(N)

In [ ]:
%time ds1 = sum(dll1)
%timeit sum(dll1)
print(ds1)

In [ ]:
print(sys.getsizeof(dll1.start), sys.getsizeof(dll1))
M1 = sys.getsizeof(dll1) + N * sys.getsizeof(dll1.start)
print('Memory footprint: %.2f Мб' % (M1/1000000))
# del ll1

In [ ]:
%time dll2 = make_dllist2(N)
%timeit make_dllist2(N)

In [ ]:
%time ds2 = sum(dll2)
%timeit sum(dll2)
print(ds2)

In [ ]:
print(sys.getsizeof(dll2.start), sys.getsizeof(dll2))
M2 = sys.getsizeof(dll2) + N * sys.getsizeof(dll2.start)
print('Memory footprint: %.2f Мб' % (M2/1000000))
# del ll2

In [ ]:
print(100*M1/M2)